### [데이터 분석]

[모듈 로딩] <hr>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import re

In [ ]:
data_path = './total.csv'

df = pd.read_csv(data_path)


In [ ]:
# 2) 풀네임 → 연도, 차이름 분리
df["연도"] = df["풀네임"].str.extract(r"(\d{4})")
df["차이름"] = df["풀네임"].str.replace(r"^\d{4}\s*", "", regex=True)


# 3) 데이터 타입 변환
numeric_cols = [ "연비_y", "가격", "주행", "거주성", "품질", "디자인", "종합점수",'연도']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # 숫자로 변환, 실패값 NaN 처리

# 4) 연도 필터링 (2014 ~ 2024)
# -----------------------------------------
df = df[df['연도'].between(2014, 2024)]

## 브랜드 추출
df["브랜드"] = df["차이름"].str.split().str[0]

# 배기량 숫자 추출
df["배기량_num"] = df["배기량"].str.extract(r"(\d+\.?\d*)").astype(float)

# 차급 규칙 함수
def classify_segment(row):

    cc = row["배기량_num"]

    # 경차
    if cc <= 1000:
        return "경차"
    
    # 소형
    if cc <= 1600:
        return "소형"
    
    # 중형
    if cc <= 2000:
        return "중형"
    
    # 대형 or 프리미엄
    return "대형/프리미엄"
df["차급"] = df.apply(classify_segment, axis=1)

## 연비_x -> 복합연비 추출
def extract_fuel(value):
    if pd.isna(value):
        return None

    # '복합 ' 뒤에 오는 첫 숫자 패턴 추출 (소수점 포함, ~ 포함 가능)
    match = re.search(r'복합\s*([0-9]+(?:\.[0-9]+)?)(?:~[0-9\.]+)?', value)

    if match:
        return float(match.group(1))
    return None

df['복합연비'] = df['연비_x'].apply(extract_fuel)

# for c in df.columns :
#     print(f'[{c}] ------- {df[c].nunique()} 개')
#     print(f'{df[c].unique()}')
# print(df['복합연비'].unique())

# df.to_csv('total2.csv')

## 컬럼 확인
display(df.columns)

In [ ]:
print(df.columns)

df = df.drop(['차종', '배기량', '연비_x'], axis=1)



In [ ]:
df.columns



In [ ]:
selected_columns = [
    # 종속변수
    '종합점수', '주행', '가격', '거주성', '품질', '디자인', '연비_y',

    # 차량 성능
    '복합연비',

    # 기술적 요소
    '연료',

    # 환경 및 메타 정보
    '연도', '차급', '브랜드', 'CO₂ 배출량'
]

df_selected = df[selected_columns].copy()



df_selected.to_csv('total3.csv')


In [ ]:
for c in df_selected.columns :
    print(f'[{c}] ------- {df_selected[c].nunique()} 개')
    print(f'{df_selected[c].unique()}')
# print(df['복합연비'].unique())

In [ ]:
## 수치형 컬럼 변경
rating_cols = ['종합점수', '주행', '가격', '거주성', '품질', '디자인', '연비_y']

# 수치형으로 변환
df_selected[rating_cols] = df_selected[rating_cols].apply(pd.to_numeric, errors='coerce')



In [ ]:
df_selected['연료'] = df_selected['연료'].replace({
    '전기, 수소': '전기'
})

# NaN → '기타'
df_selected['연료'] = df_selected['연료'].fillna('기타')

In [ ]:
## 차급 카테고리 화
df_selected['차급'] = df_selected['차급'].astype('category')

df_selected['브랜드'] = df_selected['브랜드'].astype('category')


In [ ]:
import re
import numpy as np

def clean_co2(val):
    if pd.isna(val):
        return np.nan
    
    # 범위형 -> 평균값 계산
    if "~" in val:
        nums = re.findall(r'\d+', val)
        nums = list(map(int, nums))
        return np.mean(nums)

    # 단일 숫자 추출
    match = re.search(r'\d+', val)
    return int(match.group()) if match else np.nan

df_selected['CO₂ 배출량'] = df_selected['CO₂ 배출량'].apply(clean_co2)

# 결측치 → median 대체
df_selected['CO₂ 배출량'] = df_selected['CO₂ 배출량'].fillna(df_selected['CO₂ 배출량'].median())


In [ ]:
print(df_selected.dropna(inplace=True))

df_selected.to_csv('total4.csv')

